In [5]:
from ultralytics import YOLO

# Load the YOLO12 model

import torch
import torchvision

model = YOLO('road_damage_detection_last_version.pt')

results = model(source=1, stream=True)  # generator of Results objects
print(results)  # print results to screen

<generator object BasePredictor.stream_inference at 0x13204c430>


In [6]:
import cv2
cap = cv2.VideoCapture(0)  # או נסה עם 1
if not cap.isOpened():
    print("Cannot open camera")
else:
    ret, frame = cap.read()
    if ret:
        cv2.imshow('Test', frame)
        cv2.waitKey(0)
    cap.release()
    cv2.destroyAllWindows()


2025-03-31 12:44:03.241 Python[33483:652420] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2025-03-31 12:44:05.736 Python[33483:652420] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-31 12:44:05.736 Python[33483:652420] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 

: 

In [ ]:
from ultralytics import YOLO
import cv2
import yaml

# קריאה של קובץ data.yaml
with open("hh-1/data.yaml", "r") as f:
    data_config = yaml.safe_load(f)
    
# נניח שהמבנה של data.yaml מכיל מפתח בשם "names" שבו רשימה או מילון של שמות המחלקות
# במידה וזה מילון, נהפוך אותו לרשימה לפי הסדר של המפתחות
if isinstance(data_config["names"], dict):
    # sort by key and take the values
    class_names = [data_config["names"][str(i)] for i in sorted(data_config["names"].keys(), key=lambda x: int(x))]
else:
    class_names = data_config["names"]

# טוען את המודל (וודא שהקובץ והגרסה נכונים)
model = YOLO("road_damage_detection_last_version.pt")

# קבלת זרם פריימים מהמצלמה
results = model(source=1, stream=True)

frame_count = 0  # משתנה למעקב אחרי הפריימים שנשמרו

for result in results:
    # מקבלים עותק של הפריים המקורי
    frame = result.orig_img.copy()

    # אם יש תיבות (boxes) – כלומר, זיהוי עצמים
    if result.boxes is not None and len(result.boxes) > 0:
        for box in result.boxes:
            # קבלת קואורדינטות התיבה (נניח שהפורמט הוא [x1, y1, x2, y2])
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            # המרה של המזהה של המחלקה למספר שלם
            class_idx = int(box.cls[0])
            # קבלת ערך האמון (confidence)
            confidence = box.conf[0] if hasattr(box, 'conf') else 0
            # קבלת שם המחלקה מהרשימה שקראנו מה־data.yaml
            label = class_names[class_idx] if class_idx < len(class_names) else "object"
            
            # ציור תיבת הגבולות והצבת הטקסט עם שם המחלקה ואחוז האמון
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label}: {confidence:.2f}", (x1, max(y1 - 10, 0)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # שמירת הפריים אם זוהו עצמים
        cv2.imwrite(f"detected_frame_{frame_count}.jpg", frame)
        frame_count += 1

    # הצגת הפריים עם הגרפיקה
    cv2.imshow("Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


In [3]:
from ultralytics import YOLO
model = YOLO('road_damage_detection_last_version.pt')  # Load model
model.val(data = "/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/RDD2022-3/data.yaml")  # Validate model

Ultralytics 8.3.99 🚀 Python-3.9.6 torch-2.2.2 CPU (Intel Core(TM) i7-9750H 2.60GHz)
YOLOv12n summary (fused): 159 layers, 2,558,873 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/RDD2022-3/valid/labels... 7678 images, 2309 backgrounds, 0 corrupt: 100%|██████████| 7678/7678 [00:03<00:00, 2548.24it/s]

val: WARNING ⚠️ /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/RDD2022-3/valid/images/Japan_006916_jpg.rf.5ec6ac4a0402f0f061eda3729d679fb0.jpg: 1 duplicate labels removed


val: New cache created: /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/RDD2022-3/valid/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  35%|███▌      | 169/480 [05:41<10:27,  2.02s/it]


KeyboardInterrupt: 

## הורדת ההדאטה של המודל 
# RDD2020 Dataset

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="BQvvxKrJPoAaNkv4ZXlr")
project = rf.workspace("devflo").project("rdd2022-ook7x")
version = project.version(3)
dataset = version.download("yolov12")
model = YOLO("road_damage_detection_last_version.pt")
model.val(data="RDD2022-3/data.yaml")

loading Roboflow workspace...
loading Roboflow project...
